# Prédiction des tags

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

datas = pd.read_csv("P5_06_transf_datas.csv")

datas

,stem_words,imp_words,Tags
0,convert decimal double c# want use track bar c...,convert decimal double c# want track bar chang...,c# floating-point type-conversion double ...
1,width collapse percentage width child element ...,width collapse percentage width child element ...,html css internet-explorer-7
2,calculate someone age base datetime type birth...,calculate someone age base datetime type give ...,c# .net datetime
3,calculate relative time c# give specific datet...,calculate relative time c# give specific datet...,c# datetime time datediff relative-time-s...
4,determine user timezone standard way web serve...,determine user timezone standard way web serve...,html browser timezone user-agent timezone...
...,...,...,...
29096,extract decimal part float point number extrac...,extract decimal part float point number extrac...,c floating-point decimal numbers
29097,flash load twice workaround investigate report...,flash load twice workaround investigate report...,flash swfobject
29098,etag vs header expire look around able figure ...,vs header expire look around able figure expir...,http caching etag expires-header
29099,determine longest similar portion several stri...,determine similar portion several string per t...,perl algorithm string similarity


In [2]:
samples = np.random.choice(len(datas), 5000, replace= False)
df_samples = datas.iloc[samples, :].reset_index(drop= True)

df_samples

,stem_words,imp_words,Tags
0,python leopard fink mac port python.org idiot ...,python leopard mac port break python fresh sta...,python python-2.6 python-install
1,use gcc generate assembly code intel syntax gc...,gcc generate assembly code intel syntax gcc op...,gcc x86 gnu intel assembly
2,use rspec shoulda rail set rail app finish mak...,rspec rail set rail app finish make unit test ...,ruby-on-rails ruby unit-testing testing s...
3,visual studio add reference add dll reference ...,visual studio add reference add dll reference ...,visual-studio-2008 version-control intellis...
4,c# switch string.isnullorempty possible switch...,c# switch string.isnullorempty possible switch...,c# string switch-statement semantics
...,...,...,...
4995,difference script application refer distinctio...,difference script application refer answer wri...,terminology
4996,downloadable browsable version .net framework ...,version .net framework source code source code...,.net
4997,advantage disadvantage gpgpu general purpose g...,advantage disadvantage general purpose develop...,performance optimization hpc
4998,erase elements boost ptr vector try get rid st...,erase elements boost ptr vector try get rid st...,c++ boost ptr-vector


Je sépare le jeu de données en un jeu d'entraînement et un de test.

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold

X_docs = df_samples["imp_words"]
y_docs = df_samples["Tags"]

# X_train, X_test, Y_train, Y_test = train_test_split(X_docs, y_docs, 
    # train_size= 0.7, random_state= 42)

X_train = X_docs
Y_train = y_docs

Je  transforme X et y.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vect = TfidfVectorizer(token_pattern= r'[\S]+')

x_train = tfidf_vect.fit_transform(X_train).toarray()
# x_test = tfidf_vect.transform(X_test).toarray()


In [5]:
from sklearn.decomposition import PCA

component = 1000

pca = PCA(n_components= component)
pca.fit(x_train)

pca.explained_variance_ratio_.cumsum()[-1]

0.7772525822790189

In [6]:
x_train_pca = pca.transform(x_train)
# x_test_pca = pca.transform(x_test)

In [7]:
count_vect = CountVectorizer(token_pattern= r'[\S]+', min_df= 10)
y_train = count_vect.fit_transform(Y_train).toarray()

# y_test = count_vect.transform(Y_test).toarray()


In [8]:
# X_train, X_test, Y_train, Y_test = train_test_split(x_train_pca, y_train, 
#     train_size= 0.7, random_state= 42)

In [9]:
# from sklearn.model_selection import KFold

# skf = KFold()
# for train_idx, test_idx in skf.split(x_train_pca, y_train):
#     X_train, X_test = x_train_pca[train_idx], x_train_pca[test_idx]
#     Y_train, Y_test = y_train[train_idx], y_train[test_idx]

In [10]:
def scoring_function(clf, X, y):
    y_pred = clf.predict_proba(X)

    results_preds = np.zeros(y_pred.shape)
    pred_sorted = np.sort(y_pred, axis= 1)
    diff = pred_sorted[:, -1] - np.log10(pred_sorted[:, -1] +1)
    results_preds = np.where(y_pred > 
        diff.reshape(-1, 1), 1, 0)

    return np.mean(y == results_preds)

In [11]:
def splitting(CV, clf, X, y):
    scores = []
    for train_idx, test_idx in CV.split(X, y):
        XX_train, XX_test = x_train_pca[train_idx], x_train_pca[test_idx]
        yy_train, yy_test = y[train_idx], y[test_idx]

        clf.fit(XX_train, yy_train)
        scores.append(scoring_function(clf, XX_test, yy_test))

    return np.mean(scores)

In [12]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score #, GridSearchCV, RandomizedSearchCV
# from sklearn.metrics import make_scorer
# from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit #, KFold

results = {"score": 0, "loss": "", "class_weight": "", "penalty": "", 
    "alpha": 0, "l1_ratio": 0}

CV = ShuffleSplit(n_splits= 3, train_size= 0.75, random_state= 42)

_loss = ["modified_huber", "log"] 
_penalty = ["l2", "l1", "elasticnet"]
_alpha = np.logspace(-4, 0, 5) 
_l1_ratio = np.linspace(0, 1, 5)
_class_weight = [None, "balanced"]

for loss in _loss:
    print(loss)
    for class_weight in _class_weight:
        print("\t",class_weight)
        for penalty in _penalty:
            print("\t\t", penalty)
            for alpha in _alpha:
                if penalty in ["elasticnet"]:
                    for l1_ratio in _l1_ratio:
                        clf = OneVsRestClassifier(SGDClassifier(loss= loss, class_weight= class_weight, 
                            penalty= penalty, alpha= alpha, l1_ratio= l1_ratio, early_stopping= True, 
                            random_state= 42))
                        score = splitting(CV, clf, x_train_pca, y_train)
                        if score > results["score"]:
                            results["score"] = score
                            results["loss"] = loss
                            results["class_weight"] = class_weight
                            results["penalty"] = penalty
                            results["alpha"] = alpha
                            results["l1_ratio"] = l1_ratio
                else:
                    clf = OneVsRestClassifier(SGDClassifier(loss= loss, class_weight= class_weight, 
                        penalty= penalty, alpha= alpha, early_stopping= True, random_state= 42))
                    score = splitting(CV, clf, x_train_pca, y_train)
                    if score > results["score"]:
                        results["score"] = score
                        results["loss"] = loss
                        results["class_weight"] = class_weight
                        results["penalty"] = penalty
                        results["alpha"] = alpha
                        results["l1_ratio"] = 0.15


modified_huber
	 None
		 l2
		 l1
		 elasticnet
	 balanced
		 l2
		 l1
		 elasticnet
log
	 None
		 l2
		 l1
		 elasticnet
	 balanced
		 l2
		 l1
		 elasticnet


In [13]:
results

{'score': 0.9912492131616596,
 'loss': 'modified_huber',
 'class_weight': None,
 'penalty': 'l1',
 'alpha': 0.0001,
 'l1_ratio': 0.15}

'score': 0.9919314591700134,
 'loss': 'modified_huber',
 'class_weight': None,
 'penalty': 'l1',
 'alpha': 0.0001,
 'l1_ratio': 0.15

In [15]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from sklearn.metrics import make_scorer
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import ShuffleSplit #, KFold

# pipe = Pipeline(steps=[("ovr", OneVsRestClassifier(SGDClassifier()))])

# params = {"ovr": [OneVsRestClassifier(SGDClassifier())],
#     "ovr__estimator": [SGDClassifier()], "ovr__estimator__loss": ["modified_huber", "log"], 
#     "ovr__estimator__penalty": ["l2", "l1", "elasticnet"], "ovr__estimator__alpha": np.logspace(-4, 0, 5), 
#     "ovr__estimator__l1_ratio": np.linspace(0, 1, 5), "ovr__estimator__random_state": [42], 
#     "ovr__estimator__early_stopping": [True], "ovr__estimator__class_weight": [None, "balanced"], 
#     "ovr__estimator__verbose": [2]}

# CV = ShuffleSplit(n_splits= 3, train_size= 0.75, random_state= 42)

# gcv = RandomizedSearchCV(pipe, params, cv= CV, n_jobs= -1, verbose= 1, error_score= "raise", 
#     scoring= make_scorer(scoring_function))

# gcv.fit(x_train_pca, y_train)

# print(f"Meilleurs hyperparamètres:")
# print(gcv.best_params_)
# print(f"Meilleur score:")
# print(gcv.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


TypeError: scoring_function() missing 1 required positional argument: 'y'